In [1]:
'''
python T1_BT19_Icub_joint_ae.py -k 0 -c 0 -r 1

Note: still use one-stage training with both recon loss and classification loss
'''
# Import

import os,sys
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../new_iteration/")
import pickle
import argparse
from sklearn.metrics import confusion_matrix
from datetime import datetime

from vrae.vrae import VRAEC
from vrae.utils import *
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data as data2
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime
from tas_utils_bs import get_trainValLoader, get_testLoader
import plotly


In [2]:
# # Parse argument
# parser = argparse.ArgumentParser()
# parser.add_argument("-k", "--kfold", type=int, default=0, help="kfold_number for loading data")
# parser.add_argument("-r", "--reduction", type=int, default=1, help="data reduction ratio for partial training")
# parser.add_argument("-c", "--cuda", default=0, help="index of cuda gpu to use")
# args = parser.parse_args()

In [3]:
# dummy class to replace argparser
class Args:
  kfold = 0
  reduction = 1
  cuda = '0'
  removal =0

args=Args()

In [4]:
print("load {} kfold number, reduce data to {} folds, put to cuda:{}".format(args.kfold, args.reduction, args.cuda))

# Set hyper params
kfold_number = args.kfold
data_reduction_ratio = args.reduction
removal = args.removal
shuffle = True
num_class = 20
sequence_length_B = 400
sequence_length_I = 75
number_of_features_B = 19
number_of_features_I = 60

hidden_size = 90
hidden_layer_depth = 1
latent_length = 40
batch_size = 32
learning_rate = 0.0005
n_epochs = 2000

learning_rate_2 = 0.01
n_epochs_2 = 20

dropout_rate = 0.2
cuda = True # options: True, False
print_every=30
clip = True # options: True, False
max_grad_norm=5
header_B = None
header_I = "CNN"

w_mse = 1 # mse between latent vectors
w_rB = 0.01 # recon for B
w_rI = 0.01 # recon for I
w_cB = 1 # classify for B
w_cI = 1 # classify for I


np.random.seed(1)
torch.manual_seed(1)

load 0 kfold number, reduce data to 1 folds, put to cuda:0


### Load data and preprocess

In [5]:
# Load data
data_dir = '../../new_data_folder/'

logDir = 'models_and_stat/'
# new model
model_B_joint = 'BT19_joint_ae_wrB_{}_wcB_{}_wrI_{}_wcI_{}_wC_{}_{}'\
    .format(w_rB,w_cB, w_rI, w_cI, w_mse, str(kfold_number))

model_I_joint = 'IcubCNN_joint_ae_wrB_{}_wcB_{}_wrI_{}_wcI_{}_wC_{}_{}'\
    .format(w_rB,w_cB, w_rI, w_cI, w_mse, str(kfold_number))

model_B = "BT19_ae_{}_rm_{}_wrI_{}_wC_{}_{}.pt".format(data_reduction_ratio, removal, w_rI, w_mse, str(kfold_number))
kI = 1
model_I =  "IcubCNN_ae_{}_rm_{}_wrI_{}_wC_{}_{}.pt".format(data_reduction_ratio, removal, w_rI, w_mse, str(kI))

device = torch.device("cuda:{}".format(args.cuda))
print("Loading data...")

train_loader, val_loader, train_dataset, val_dataset = get_trainValLoader(data_dir, k=kfold_number, spike_ready=False, batch_size=batch_size, shuffle=shuffle)
test_loader, test_dataset = get_testLoader(data_dir, spike_ready=False, batch_size=batch_size, shuffle=shuffle)

Loading data...


In [8]:
joint_dict_name = "Joint-dict.pkl"
mB_dict_name = model_B[:-3] + "_stats.pkl"
mI_dict_name = model_I[:-3] + "_stats.pkl"
print(mB_dict_name)

BT19_ae_1_rm_0_wrI_0.01_wC_1_0_stats.pkl


In [7]:
joint_dict = pickle.load(open(logDir+joint_dict_name, 'rb'))
print(joint_dict.keys())

FileNotFoundError: [Errno 2] No such file or directory: 'models_and_stat/Joint-dict.pkl'

In [9]:
mB_dict = pickle.load(open(logDir+mB_dict_name, 'rb'))
mI_dict = pickle.load(open(logDir+mI_dict_name, 'rb'))
print(mB_dict.keys())

FileNotFoundError: [Errno 2] No such file or directory: 'models_and_stat/BT19_ae_1_rm_0_wrI_0.01_wC_1_0_stats.pkl'

In [ ]:
# assert n_epochs == len(epoch_train_acc), "different epoch length {} {}".format(n_epochs, len(epoch_train_acc))
print(len(joint_dict['epoch_train_acc_B']), len(mB_dict['epoch_train_acc']))
fig, ax = plt.subplots(figsize=(15, 7))
# ax.plot(np.arange(200), joint_dict['epoch_train_acc_B'][:200], label="joint_B_train_acc")
# ax.plot(np.arange(200), mB_dict['epoch_train_acc'][:200], label="B_train_acc")
# ax.plot(np.arange(200), joint_dict['epoch_train_acc_I'][:200], label="joint_I_train_acc")
# ax.plot(np.arange(200), mI_dict['epoch_train_acc'][:200], label="I_train_acc")

ax.plot(np.arange(200), joint_dict['epoch_val_acc_B'][:200], label="joint_B_val_acc")
ax.plot(np.arange(200), mB_dict['epoch_val_acc'][:200], label="B_val_acc")
ax.plot(np.arange(200), joint_dict['epoch_val_acc_I'][:200], label="joint_I_val_acc")
ax.plot(np.arange(200), mI_dict['epoch_val_acc'][:200], label="I_val_acc")


ax.set_xlabel('epoch')
ax.set_ylabel('acc')
ax.grid(True)
plt.legend(loc='upper right')
figname = logDir +"T124_val_acc_comp.png"
plt.savefig(figname)
plt.show()
